<img src="../assets/CoLLIE_blue.png" alt="GoLLIE" width="200"/>

# Named Entity Recognition with GoLLIE

This notebook is an example of how to run Named Entity Recognition with GoLLIE. This notebook covers:

- How to define the guidelines for a task
- How to load GoLLIE
- How to generate model inputs
- How to parse the output
- How to implement a scorer and evaluate the output

You can modify this notebook to run any Named Entity Recognition task you want

### Import requeriments

See the requeriments.txt file in the main directory to install the required dependencies

In [1]:
import sys
sys.path.append("../") # Add the GoLLIE base directory to sys path

In [2]:
import rich
import logging
from src.model.load_model import load_model
import black
import inspect
from jinja2 import Template
import tempfile
from src.tasks.utils_typing import AnnotationList
logging.basicConfig(level=logging.INFO)
from typing import Dict, List, Type

/hhome/nlp2_g09/miniconda3/envs/Gollie/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load GoLLIE

We will load GOLLIE-7B from the huggingface-hub.
You can use the function AutoModelForCausalLM.from_pretrained if you prefer it. However, we provide a handy load_model function with many functionalities already implemented that will assist you in reproducing our results.

Please note that setting use_flash_attention=True is mandatory. Our flash attention implementation has small numerical differences compared to the attention implementation in Huggingface. Using use_flash_attention=False will result in the model producing inferior results. Flash attention requires an available CUDA GPU. Running GOLLIE pre-trained models on a CPU is not supported. We plan to address this in future releases.

- Set force_auto_device_map=True to automatically load the model on available GPUs.
- Set quantization=4 if the model doesn't fit in your GPU memory.

In [3]:
model, tokenizer = load_model(
    inference=True,
    model_weights_name_or_path="HiTZ/GoLLIE-7B",
    quantization=None,
    use_lora=False,
    force_auto_device_map=True,
    use_flash_attention=True,
    torch_dtype="bfloat16"
)

INFO:root:Loading model model from HiTZ/GoLLIE-7B
INFO:root:We will load the model using the following device map: auto and max_memory: None
/hhome/nlp2_g09/miniconda3/envs/Gollie/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO:root:Loading model with dtype: torch.bfloat16


>>>> Flash Attention installed


>>>> Flash RoPE installed


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/hhome/nlp2_g09/miniconda3/envs/Gollie/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  8.78it/s]
INFO:root:Model dtype: torch.bfloat16
INFO:root:Total model memory footprint: 13477.101762 MB


## Define the guidelines

First, we will define the labels and guidelines for the task. We will represent them as Python classes.

The following guidelines have been defined for this example. They were not part of the pre-training dataset. Therefore, we will run GOLLIE in zero-shot settings using unseen labels.

💡 Be creative and try to define your own guidelines to test GoLLIE!

In [4]:
from typing import List

from src.tasks.utils_typing import Entity, dataclass

"""
Entity definitions
"""


@dataclass
class PrivateSpaceCompany(Entity):
    """Refers to private companies primarily focused on space exploration, transportation, 
    satellite launch, or space-based services. These are non-governmental entities that have 
    a commercial interest in space activities."""

    span: str  # Such as: "Blue origin", "Boeing", "Northrop Grumman", "Arianespace"


@dataclass
class PublicSpaceCompany(Entity):
    """Refers to governmental entities or agencies that are primarily focused on space 
    exploration, research, transportation, satellite launch, or other space-based services. 
    These entities are state-owned and operated and are generally funded through public funds."""

    span: str  # Such as "ESA", "ISRO", "CNSA"


@dataclass
class Planet(Entity):
    """Refers to celestial bodies that orbit a star. Planets are large enough 
    to have cleared their orbits of other debris and have a nearly round shape 
    due to their self-gravity."""

    span: str  # Such as: "Earth", "Jupiter", "Venus", "Mercury", "Saturn"


@dataclass
class Launcher(Entity):
    """Refers to a vehicle designed primarily to transport payloads from the Earth's 
    surface to space. Launchers can carry various payloads, including satellites, 
    crewed spacecraft, and cargo, into various orbits or even beyond Earth's orbit. 
    They are usually multi-stage vehicles that use rocket engines for propulsion."""

    span: str  # Such as: "Sturn V", "Atlas V", "Soyuz", "Ariane 5"



ENTITY_DEFINITIONS: List[Entity] = [
    PrivateSpaceCompany,
    PublicSpaceCompany,
    Planet,
    Launcher,
]
    
if __name__ == "__main__":
    cell_txt = In[-1]

### Print the guidelines to guidelines.py

Due to IPython limitations, we must write the content of the previous cell to a file and then import the content from that file.

In [5]:
with open("guidelines.py","w",encoding="utf8") as python_guidelines:
    print(cell_txt,file=python_guidelines)

from guidelines import *

We use inspect.getsource to get the guidelines as a string

In [6]:
guidelines = [inspect.getsource(definition) for definition in ENTITY_DEFINITIONS]

## Define input sentence

Here we define the input sentence and the gold labels.

You can define and empy list as gold labels if you don't have gold annotations.

In [7]:
text = "SpaceX is colaborating with NASA in the mission to bring humans to Mars using their new Starship rocket."
gold = [
    PrivateSpaceCompany(span="SpaceX"),
    PublicSpaceCompany(span="NASA"),
    Planet(span="Mars"),
    Launcher(span="Starship")
]

## Filling a template

For NER we will use the following prompt template.
We use Jinja templates, which are easy to implement and exceptionally fast. For more information, visit: https://jinja.palletsprojects.com/en/3.1.x/api/#high-level-api.

```Python
# The following lines describe the task definition
{%- for definition in guidelines %}
{{ definition }}
{%- endfor %}

# This is the text to analyze
text = {{ text.__repr__() }}

# The annotation instances that take place in the text above are listed here
result = [
{%- for ann in annotations %}
    {{ ann }},
{%- endfor %}
]

```

This template is stored in `templates/prompt.txt`

In [8]:
# Read template
with open("../templates/prompt.txt", "rt") as f:
    template = Template(f.read())
# Fill the template
formated_text = template.render(guidelines=guidelines, text=text, annotations=gold, gold=gold)

### Black Code Formatter

We use the Black Code Formatter to automatically unify all the prompts to the same format. 

https://github.com/psf/black

In [9]:
black_mode = black.Mode()
formated_text = black.format_str(formated_text, mode=black_mode)

### Print the filled and formatted template

In [10]:
rich.print(formated_text)

# The following lines describe the task definition
@dataclass
class PrivateSpaceCompany(Entity):
    """Refers to private companies primarily focused on space exploration, transportation,
    satellite launch, or space-based services. These are non-governmental entities that have
    a commercial interest in space activities."""

    span: str  # Such as: "Blue origin", "Boeing", "Northrop Grumman", "Arianespace"


@dataclass
class PublicSpaceCompany(Entity):
    """Refers to governmental entities or agencies that are primarily focused on space
    exploration, research, transportation, satellite launch, or other space-based services.
    These entities are state-owned and operated and are generally funded through public funds.
    """

    span: str  # Such as "ESA", "ISRO", "CNSA"


@dataclass
class Planet(Entity):
    """Refers to celestial bodies that orbit a star. Planets are large enough
    to have cleared their orbits of other debris and have a nearly round shape
    due to their self-gravity."""

    span: str  # Such as: "Earth", "Jupiter", "Venus", "Mercury", "Saturn"


@dataclass
class Launcher(Entity):
    """Refers to a vehicle designed primarily to transport payloads from the Earth's
    surface to space. Launchers can carry various payloads, including satellites,
    crewed spacecraft, and cargo, into various orbits or even beyond Earth's orbit.
    They are usually multi-stage vehicles that use rocket engines for propulsion."""

    span: str  # Such as: "Sturn V", "Atlas V", "Soyuz", "Ariane 5"


# This is the text to analyze
text = "SpaceX is colaborating with NASA in the mission to bring humans to Mars using their new Starship rocket."

# The annotation instances that take place in the text above are listed here
result = [
    PrivateSpaceCompany(span="SpaceX"),
    PublicSpaceCompany(span="NASA"),
    Planet(span="Mars"),
    Launcher(span="Starship"),
]

## Prepare model inputs

We remove everything after `result =` to run inference with the model.

In [11]:
prompt, _ = formated_text.split("result =")
prompt = prompt + "result ="

Tokenize the input sentence

In [12]:
model_input = tokenizer(prompt, add_special_tokens=True, return_tensors="pt")

Remove the `eos` token from the input

In [13]:
model_input["input_ids"] = model_input["input_ids"][:, :-1]
model_input["attention_mask"] = model_input["attention_mask"][:, :-1]

## Run GoLLIE

We generate the predictions using GoLLIE.

We use `num_beams=1` and `do_sample=False` in our exmperiments. But feel free to experiment with differen decoding strategies 😊

In [14]:
%%time

model_ouput = model.generate(
    **model_input.to(model.device),
    max_new_tokens=128,
    do_sample=False,
    min_new_tokens=0,
    num_beams=1,
    num_return_sequences=1,
)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RuntimeError: invalid argument to exchangeDevice

### Print the results

In [15]:
for y, x in enumerate(model_ouput):
    print(f"Answer {y}")
    rich.print(tokenizer.decode(x,skip_special_tokens=True).split("result = ")[-1])

NameError: name 'model_ouput' is not defined

## Parse the output

The output is a Python list of instances, we can execute it  🤯

We define the AnnotationList class to parse the output with a single line of code. The `AnnotationList.from_output` function filters any label that we did not define (hallucinations) to prevent getting an `undefined class` error. 

In [ ]:
result = AnnotationList.from_output(
    tokenizer.decode(model_ouput[0],skip_special_tokens=True).split("result = ")[-1],
    task_module="guidelines"
    )
rich.print(result)

[
    PrivateSpaceCompany(span='SpaceX'),
    PublicSpaceCompany(span='NASA'),
    Planet(span='Mars'),
    Launcher(span='Starship')
]

Labels are an instance of the defined classes:

In [ ]:
type(result[0])

guidelines.PrivateSpaceCompany

In [ ]:
result[0].span

'SpaceX'

# Evaluate the result

Finally, we will evaluate the outputs from the model.

First, we define an Scorer, for Named Entity Recognition, we will use the `SpanScorer` class.

We need to define the `valid_types` for the scorer, which will be the labels that we have defined. 

In [ ]:
from src.tasks.utils_scorer import SpanScorer

class MyEntityScorer(SpanScorer):
    """Compute the F1 score for Named Entity Recogtion Tasks"""

    valid_types: List[Type] = ENTITY_DEFINITIONS

    def __call__(self, reference: List[Entity], predictions: List[Entity]) -> Dict[str, Dict[str, float]]:
        output = super().__call__(reference, predictions)
        return {"entities": output["spans"]}


### Instanciate the scorer

In [ ]:
scorer = MyEntityScorer()

### Compute F1 

In [ ]:
scorer_results = scorer(reference=[gold],predictions=[result])
rich.print(scorer_results)

{
    'entities': {
        'precision': 1.0,
        'recall': 1.0,
        'f1-score': 1.0,
        'class_scores': {
            'PrivateSpaceCompany': {
                'tp': 1,
                'total_pos': 1,
                'total_pre': 1,
                'precision': 1.0,
                'recall': 1.0,
                'f1-score': 1.0
            },
            'PublicSpaceCompany': {
                'tp': 1,
                'total_pos': 1,
                'total_pre': 1,
                'precision': 1.0,
                'recall': 1.0,
                'f1-score': 1.0
            },
            'Planet': {'tp': 1, 'total_pos': 1, 'total_pre': 1, 'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0},
            'Launcher': {
                'tp': 1,
                'total_pos': 1,
                'total_pre': 1,
                'precision': 1.0,
                'recall': 1.0,
                'f1-score': 1.0
            }
        }
    }
}

GoLLIE has successfully labeled a sentence using a set of labels that were not part of the pretraining dataset 🎉🎉🎉

GoLLIE will perform well on labels with well-defined and clearly bounded guidelines. 

Please share your cool experiments with us; we'd love to see what everyone is doing with GoLLIE!
- [@iker_garciaf](https://twitter.com/iker_garciaf)
- [@osainz59](https://twitter.com/osainz59)